In [14]:
import pandas as pd
loc = "C:\\Users\\jayan\\OneDrive\\Documents\\books.csv"
df = pd.read_csv(loc)

In [15]:
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1.0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.780000e+12,eng,652.0,2095690.0,27591.0,9/16/2006,Scholastic Inc.
1,2.0,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.780000e+12,eng,870.0,2153167.0,29221.0,9/1/2004,Scholastic Inc.
2,4.0,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.780000e+12,eng,352.0,6333.0,244.0,11/1/2003,Scholastic
3,5.0,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9.780000e+12,eng,435.0,2339585.0,36325.0,5/1/2004,Scholastic Inc.
4,8.0,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,439682584,9.780000e+12,eng,2690.0,41428.0,164.0,9/13/2004,Scholastic


In [16]:
df.shape

(11127, 12)

In [17]:
df.columns
df.dtypes

bookID                float64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                float64
language_code          object
  num_pages           float64
ratings_count         float64
text_reviews_count    float64
publication_date       object
publisher              object
dtype: object

In [18]:
df.isnull().sum()

bookID                4
title                 4
authors               4
average_rating        4
isbn                  4
isbn13                4
language_code         4
  num_pages           4
ratings_count         4
text_reviews_count    4
publication_date      4
publisher             4
dtype: int64

In [21]:
df = df.dropna().reset_index(drop=True)

In [22]:
df.isnull().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [23]:
df.shape

(11123, 12)

In [24]:
df['publication_date'] = pd.to_datetime(df['publication_date'])

In [30]:
df.dtypes

bookID                       float64
title                         object
authors                       object
average_rating               float64
isbn                          object
isbn13                       float64
language_code                 object
  num_pages                  float64
ratings_count                float64
text_reviews_count           float64
publication_date      datetime64[ns]
publisher                     object
dtype: object

In [32]:
df.columns = df.columns.str.replace(' ', '')

In [33]:
df1 = df[['bookID','title','authors','average_rating']].copy()
df2 = df[['bookID','isbn','isbn13','language_code','num_pages']].copy()
df3 = df[['bookID','ratings_count','text_reviews_count','publication_date','publisher']].copy()

In [39]:
print('df shape',df.shape)
print('df1 shape',df1.shape)
print('df2 shape',df2.shape)
print('df3 shape',df3.shape)

df shape (11123, 12)
df1 shape (11123, 4)
df2 shape (11123, 5)
df3 shape (11123, 5)


In [57]:
import psycopg2
import json

In [58]:
with open(r'C:\Users\jayan\OneDrive\Documents\db_cred.json','r') as j:
    cred = json.load(j)

In [75]:
from sqlalchemy import create_engine
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(cred["user"],cred["password"], cred["host"], cred['port'], cred["database"])
db = create_engine(db_string)

In [79]:
df1.to_sql('df1',db, if_exists= 'replace',index=True)

In [80]:
df2.to_sql('df2',db, if_exists= 'replace',index=True)

In [81]:
df3.to_sql('df3',db, if_exists= 'replace',index=True)

In [142]:
with db.connect() as conn:
    statement = '''SELECT df1."bookID", df1."title"
FROM df1 INNER JOIN df3 ON df1."bookID" = df3."bookID" ORDER BY "publication_date" ASC limit 5 offset 0'''
    result = conn.execute(statement)
    for i,row in enumerate(result):
        print("Top {0} oldest book is {1}".format(i+1,row["title"]))

Top 1 oldest book is Consider the Lilies
Top 2 oldest book is On Duties (De Officiis)
Top 3 oldest book is Agricola / Germania / Dialogue on Oratory
Top 4 oldest book is History of the Peloponnesian War: Bk. 1-2
Top 5 oldest book is History of the Peloponnesian War: Bk. 5-6


In [138]:
with db.connect() as conn:
    statement = '''SELECT df2."language_code", AVG(df1."average_rating")
FROM df1 INNER JOIN df2 ON df1."bookID" = df2."bookID" GROUP BY df2."language_code" ORDER BY AVG(df1."average_rating") DESC LIMIT 5 OFFSET 0'''
    result = conn.execute(statement)
    for row in result:
        print("Average Rating for Language code {0} is {1}".format(row[0],row[1]))

Average Rating for Language code wel is 5.0
Average Rating for Language code gla is 4.47
Average Rating for Language code zho is 4.456428571428571
Average Rating for Language code tur is 4.42
Average Rating for Language code ale is 4.36


In [144]:
with db.connect() as conn:
    statement = '''SELECT df1."authors", SUM(df3."ratings_count")
FROM df1 INNER JOIN df3 ON df1."bookID" = df3."bookID" GROUP BY df1."authors" ORDER BY SUM(df3."ratings_count") DESC LIMIT 5 OFFSET 0'''
    result = conn.execute(statement)
    for i,row in enumerate(result):
        print("Top {0} rated author is {1} with ratings {2}".format(i+1,row[0],row[1]))

Top 1 rated author is J.K. Rowling/Mary GrandPré with ratings 8923980.0
Top 2 rated author is J.R.R. Tolkien with ratings 4776638.0
Top 3 rated author is Stephenie Meyer with ratings 4597666.0
Top 4 rated author is Dan Brown with ratings 4135380.0
Top 5 rated author is Nicholas Sparks with ratings 3048149.0


In [147]:
with db.connect() as conn:
    statement = '''SELECT count(DISTINCT df2."language_code"), df3."publisher"
FROM df2 INNER JOIN df3 ON df2."bookID" = df3."bookID" GROUP BY df3."publisher" ORDER BY count(DISTINCT df2."language_code") DESC LIMIT 5 OFFSET 0'''
    result = conn.execute(statement)
    for i,row in enumerate(result):
        print("Top {0} diverse publisher is {1} with books published in {2} languages".format(i+1,row[1],row[0]))

Top 1 diverse publisher is Oxford University Press  USA with books published in 6 languages
Top 2 diverse publisher is Dover Publications with books published in 5 languages
Top 3 diverse publisher is Mariner Books with books published in 5 languages
Top 4 diverse publisher is HarperOne with books published in 4 languages
Top 5 diverse publisher is Cambridge University Press with books published in 4 languages
